In [1]:
import pandas as pd
import numpy as np

### 1. Постановка задачи

Имеется файл 2 с различными оценками асессоров.

Оценки могут принимать значение [0, 1], т.е. задание, которое сделали асессоры, имеет бинарную шкалу.

Используя данные об оценках, установите, какие асессоры хуже всего справились с заданием.
* На какие показатели вы ориентировались и какие метрики вы использовали для ответа на этот вопрос? 
* Можно ли предложить какие-то новые метрики для подсчета качества асессоров с учетом природы оценок у этого бинарного задания?

### 2. Понимание данных

Формат файла: login tuid docid jud cjud; разделитель — табуляция \t

Пояснение к формату: 
* login — логин асессора
* uid — id асессора (user id)
* docid — id оцениваемого документа (document id)
* jud — оценка асессора (judgement)
* cjud — правильная оценка (correct judgement)

Пропущенных значений в данных нет.

Нулевые значения есть в столбцах uid, docid, jud, cjud - в данном случае нули выступают частью данных и не являются ошибкой.

In [2]:
df = pd.read_csv('two.csv', '\t')

In [3]:
df.head(20)

,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0
5,assessor123,123,1,1,1
6,assessor491,491,1,0,1
7,assessor409,409,1,1,1
8,assessor396,396,1,1,1
9,assessor295,295,1,1,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   login   250000 non-null  object
 1   uid     250000 non-null  int64 
 2   docid   250000 non-null  int64 
 3   jud     250000 non-null  int64 
 4   cjud    250000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 9.5+ MB


In [5]:
for i in df.columns:
    if len(df[df[i] == 0]) != 0:
        print(f'Кол-во значений равных нулю в столбце {i}:', len(df[df[i] == 0]))
    else: print(f'Значений равных нулю в столбце {i} - нет')

Значений равных нулю в столбце login - нет
Кол-во значений равных нулю в столбце uid: 401
Кол-во значений равных нулю в столбце docid: 5
Кол-во значений равных нулю в столбце jud: 190470
Кол-во значений равных нулю в столбце cjud: 220020


### 3. Подготовка данных

Данные готовы к проведению анализа и не требуют дополнительных преобразований

### 4. Анализ данных

Для решения задачи сравним ответы ассессоров с правильными. Затем посчитаем количество верных и ошибочных ответов для каждого ассессора, вычислим общее количество выполненных ответов и их процент верности. Все это позволит определить ассессоров, которые хуже всего справились с работой.

In [9]:
df.head()

,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0


In [125]:
# считаем количество верных и ошибочных ответов для каждого ассессора

df['check'] = df['jud'] == df['cjud']

assessor_check = pd.pivot_table(data=df[['login', 'docid', 'check']],
                                index=['login'],
                                columns=['check'],
                                aggfunc='count')

assessor_check.columns = ['false', 'true']
assessor_check.reset_index(inplace=True)

In [126]:
assessor_check

,login,false,true
0,assessor0,65,336
1,assessor1,82,330
2,assessor10,92,351
3,assessor100,83,334
4,assessor101,50,353
...,...,...,...
595,assessor95,39,389
596,assessor96,88,363
597,assessor97,40,383
598,assessor98,53,397


In [127]:
# вычисляем общее количнство выполненных заданий и процент верных ответов

assessor_check['cnt_ts'] = assessor_check['true'] + assessor_check['false']

assessor_check['ratio_true'] = assessor_check['true'] * 100 / assessor_check['cnt_ts']

### Вывод

Опираясь на метрики "процент верности выполнения задания" и "количество выполненных заданий", получаем список 5 ассессоров, которые хуже всех справились с заданием

Для подсчета качества ассессоров можно дополнительно учитывать:
* время выполнения задания
* сложность задания

Это позволит более корректно выявлять "лучших" и "худших" исполнителей

In [135]:
assessor_check[['login', 'ratio_true', 'cnt_ts']].groupby(['ratio_true', 'cnt_ts']).min().head()

,,login
ratio_true,cnt_ts,
42.579075,411,assessor56
46.009390,426,assessor3
47.570332,391,assessor118
48.058252,412,assessor390
48.484848,99,assessor234
